In [6]:
import os
from openai import OpenAI


In [16]:

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "give me a story about two monkeeys and it should be about 300 words"},

  ],
  stream=True
)

In [15]:
response.choices[0].message.content

"Once upon a time, in the heart of a lush tropical forest, there lived two mischievous monkeys named Milo and Luna. They were inseparable, swinging from tree to tree, playing pranks on other animals, and exploring every inch of their vibrant home.\n\nMilo, with his playful brown eyes and quick wit, was always the leader of their escapades, while Luna, with her soft fur and gentle nature, kept him grounded and out of trouble. They were the perfect pair, bringing laughter and joy to all the creatures in the forest.\n\nOne sunny morning, Milo and Luna stumbled upon a magnificent mango tree deep in the forest. Its branches were heavy with ripe, juicy fruits, glistening in the sunlight. The monkeys' eyes widened with excitement, as they knew this was a treasure they couldn't resist.\n\nMilo proposed a daring plan to gather as many mangoes as they could carry and share them with their friends. Luna, always the voice of reason, warned him about the consequences of stealing from the tree witho

In [17]:
for chunk in response:
    # print(chunk)
    print(chunk.choices[0].delta.content)
    # print("****************")


Once
 upon
 a
 time
,
 in
 the
 heart
 of
 a
 lush
 jungle
,
 there
 lived
 two
 monkeys
 named
 Oliver
 and
 Mia
.
 They
 were
 the
 best
 of
 friends
,
 spending
 their
 days
 swinging
 through
 the
 trees
,
 playing
 games
,
 and
 eating
 delicious
 fruits
 together
.


One
 sunny
 morning
,
 Oliver
 and
 Mia
 stumbled
 upon
 a
 mysterious
 tree
 filled
 with
 the
 most
 beautiful
,
 shiny
 bananas
 they
 had
 ever
 seen
.
 Exc
ited
ly
,
 they
 decided
 to
 make
 it
 their
 mission
 to
 get
 to
 the
 top
 of
 the
 tree
 and
 have
 a
 taste
 of
 those
 tantal
izing
 bananas
.


Ol
iver
,
 being
 more
 agile
 and
 quick
,
 volunteered
 to
 climb
 the
 tree
 first
.
 With
 great
 determination
 and
 skill
,
 he
 made
 his
 way
 up
 the
 trunk
,
 using
 his
 long
 arms
 to
 swing
 from
 branch
 to
 branch
.
 Mia
 cheered
 him
 on
 from
 below
,
 eagerly
 awaiting
 her
 turn
 to
 reach
 the
 top
.


As
 Oliver
 ne
ared
 the
 bunch
 of
 bananas
,
 he
 found
 himself
 in
 a
 tricky
 situa

In [1]:
from langchain.llms import OpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [4]:
from dotenv import load_dotenv


load_dotenv()

# dburi = os.getenv("DATABASE_URL")
dburi = "sqlite:///academy.db"
db = SQLDatabase.from_uri(dburi)
llm = OpenAI(temperature=0)

db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

db_chain.run("How many rows is in the responses table of this db?")
db_chain.run("Describe the responses table")
db_chain.run("What are the top 3 countries where these responses are from?")
db_chain.run("Give me a summary of how these customers come to hear about us. \
    What is the most common way they hear about us?")

ArgumentError: Could not parse SQLAlchemy URL from string 'academy.db'

In [5]:
import os
import sqlite3
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

# Load environment variables from .env file
load_dotenv()

# Set up the database URI
dburi = "sqlite:///academy.db"

# Create the SQLite database and table if they don't exist
def setup_database():
    conn = sqlite3.connect('academy.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS responses (
        id INTEGER PRIMARY KEY,
        query TEXT NOT NULL,
        response TEXT NOT NULL,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    conn.commit()
    conn.close()

setup_database()

# Connect to the database using langchain's SQLDatabase
db = SQLDatabase.from_uri(dburi)

# Set up the OpenAI LLM
llm = OpenAI(temperature=0)

# Define the SQLDatabaseChain
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

# Function to run a query and store the response in the database
def run_query_and_store_response(query):
    response = db_chain.run(query)
    conn = sqlite3.connect('academy.db')
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO responses (query, response) VALUES (?, ?)
    ''', (query, response))
    conn.commit()
    conn.close()
    return response

# Running queries and storing responses
response_1 = run_query_and_store_response("How many rows are in the responses table of this db?")
response_2 = run_query_and_store_response("Describe the responses table")
response_3 = run_query_and_store_response("What are the top 3 countries where these responses are from?")
response_4 = run_query_and_store_response("Give me a summary of how these customers come to hear about us. What is the most common way they hear about us?")

# Print the responses
print(response_1)
print(response_2)
print(response_3)
print(response_4)


d:\Work\Python\Streamlit\.venv\lib\site-packages\langchain_experimental\sql\base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new SQLDatabaseChain chain...
How many rows are in the responses table of this db?
SQLQuery:SELECT COUNT(*) FROM responses
SQLResult: [(0,)]
Answer:0
> Finished chain.


> Entering new SQLDatabaseChain chain...
Describe the responses table
SQLQuery:SELECT "query", response, timestamp FROM responses LIMIT 5
SQLResult: [('How many rows are in the responses table of this db?', '0', '2024-06-21 10:43:51')]
Answer:The responses table contains the following columns: id (INTEGER), query (TEXT), response (TEXT), and timestamp (DATETIME). The table has 3 rows, with the first row containing the query "How many rows are in the responses table of this db?", the response "0", and the timestamp "2024-06-21 10:43:51".
> Finished chain.


> Entering new SQLDatabaseChain chain...
What are the top 3 countries where these responses are from?
SQLQuery:SELECT response, COUNT(*) AS count FROM responses GROUP BY response ORDER BY count DESC LIMIT 3
SQLResult: [('The responses table contains the 